In [ ]:
#Download all files from the links saved in a txt file

import urllib.request
import zipfile
from pathlib import Path
import os

urls=[]
dest_Path='destination'

with open('open sourcefile') as f:
    for line in f:
        urls.append(line)

    for link in urls:
            link = link.strip()
            name = link.rsplit('/', 1)[-1]
            filename = os.path.join(dest_Path, name)
            
            if not os.path.isfile(filename):
                print('Downloading: ' + filename)
            try:
                urllib.request.urlretrieve(link, filename)
            except Exception as inst:
                print(inst)
                print('  Encountered unknown error. Continuing.')

In [ ]:

#Read data from csv_file and complete cleaning taks on it

import pandas as pd
from pandas.io.json import json_normalize
from pathlib import Path
import os,json
import pandas.api.types as ptypes
import nltk
from nltk import pos_tag, sent_tokenize, wordpunct_tokenize
from nltk.tokenize import MWETokenizer
import numpy as np
import re
import string
import itertools
from sklearn.feature_extraction.text import CountVectorizer

def getDataFromJsonToCsv(pathFrom,pathTo):

    # set path to file
    p = Path(pathFrom)
    #save data from json file
    data=[]

    
    rem = string.punctuation
    rem=rem.translate({ord('_'): None})
    pattern = r"[{}]".format(rem)
    
    # read json and save in data variable
    with p.open('r', encoding='utf-8') as f:
        data = [json.loads(line) for line in f]

    # create and normalize dataframe
    df = json_normalize(data,'entities',['date',"url","lang"],record_prefix='')
    
    url_values = df[(df['name'].str.lower().str.contains('facebook|economy|amazon|microsoft|google|apple')) & (df['avgSalience']>0.04)].url.unique()

    df=df[df.url.isin(url_values)]
    
    #Change whitespace into an underscore, using numpy to optimize the scaling
    df['name']=np.core.defchararray.replace(df['name'].values.astype(str), ' ', '_')
    
    
    
    #Group date by url/date and sum the single rows of name/numMentions in single list rows
    df=df.groupby(['url','date'], as_index=False).agg({
    'name': lambda x: list(x),
    'numMentions': lambda x: list(x),
    })
    
    #Nehme die Liste auseinander und ordne sie in reihen
    series = [df[i].explode() for i in ['name','numMentions']]
    #multipliziere die jeweilihen miteinander und füge sie in die Liste
    df['name'] = series[0].str.lower().repeat(series[1]).groupby(level=0).agg(list)
    
    #digits into blank
    series_one = [df[i].explode() for i in ['name']]
    df['name']=series_one[0].str.replace('\w*\d\w*','').groupby(level=0).agg(list)
    
    #punctation into blank
    series_two = [df[i].explode() for i in ['name']]
    df['name']=series_two[0].str.replace(pattern,'').groupby(level=0).agg(list)
    
    df.to_csv(pathTo)
    
    return df

In [ ]:
#Plot table of Gdelt Data

import matplotlib.pyplot as plt
import pandas as pd
from pandas.plotting import table 


fig, ax = plt.subplots(figsize=(12, 2)) 
ax.xaxis.set_visible(False)  
ax.yaxis.set_visible(False)  
ax.set_frame_on(False)  k
table = table(ax, data, loc='upper right')  
table.auto_set_font_size(False) 
table.set_fontsize(12) 
table.scale(4, 4) 
table.auto_set_column_width(col=list(range(len(data.columns))))
plt.savefig("save_to.png",dpi=800, bbox_inches = "tight")

In [ ]:
#Vectorize the dataFrame to explore it easier 

#Vectorize the corpus with sklearn Countvectorizer
def Countvectorize_corpus(data):
  
    vectorizer = CountVectorizer(tokenizer=lambda doc: doc, lowercase=False)
    bag_of_words = vectorizer.fit_transform(data['name'])
    X=bag_of_words.toarray()
    feature_names = vectorizer.get_feature_names()
    return pd.DataFrame(bag_of_words.toarray(), columns=feature_names)


#wörter suchen
def search_word(vec,bag,word):
    i = vec.get_feature_names().index(word)
    return bag[:,i].nonzero()[0]
#search_word(vectorizer,bag_of_words,'linear')

#merge original dataframe with vectorized one
def Vectorized_dataframe(data):
    data['date']=pd.to_datetime(data['date'].astype(float), unit='s')
    data = data.merge(test, left_index=True, right_index=True, how='left')
    data.drop(['name_x','numMentions'],axis=1)
    return dd
    

In [ ]:
Countvectorize_corpus(master_file.sample(5))

In [ ]:
#Read all downloaded json Files, convert them into csv_files and concat all files into one master file

import os
import pandas as pd
def queryFiles(data):
    
    json_files=[]
    csv_files=[]
    count=1615
    #read all json files in a directonary
    json_files = [pos_json for pos_json in os.listdir(data) if pos_json.endswith('json')]
    
    #if there were any readable json files, optimize and convert it to csv
    if len(json_files)>0:
        for i in range(len(json_files)):
            csv_files.append(getDataFromJsonToCsv(data+json_files[i],'sourcefile//file{}.csv'.format(count)))
            count+=1
            
       
    new_df= pd.concat(csv_files)
    
    return new_df.to_csv("save_to")
      
queryFiles('jsonFiles') 

In [ ]:
#Create own Lexicon based on LM and Minqing Hu and Bing Liu lexicons by splitting the words into positve/negative lists

from fractions import Fraction
lm_master_dict=pd.read_csv('loghranmcdonald dictionary')
positive_words=[]
negative_words=[]


n_frame=lm_master_dict[lm_master_dict['Negative']>0]
p_frame=lm_master_dict[lm_master_dict['Positive']>0]

positive_words=[x for x in p_frame['Word']]
negative_words=[x for x in n_frame['Word']]

negative=0
positive=0
result=0

#Opinion Lexicon from Minqing Hu and Bing Liu
with open("lui dictionary") as f:
    for line in f:
        newline = line.rstrip('\n')
        if newline.upper() not in positive_words:
            positive_words.append(newline.upper())

len(positive_words),len(negative_words)

In [ ]:
#Train own lexicon on labelled data based on Full-Economic-News-DFE-83986 and Headline Data
#Normalize and cleane the labelled data, fill empty rows with values to avoid exceptions

import ast
from fractions import Fraction
from itertools import zip_longest

p=Path("Headline_Trainingdata.json")
news=pd.read_json(p)

mmm=pd.read_csv('Full-Economic-News-DFE-839861.csv')
mmm[mmm['positivity']>0]
mmm=mmm.loc[:, mmm.columns.intersection(['positivity','headline','text'])]
mmm[mmm['positivity']>4].iloc[0][2]

news=news.append(mmm[mmm['positivity']>0], ignore_index=True)
news[news['positivity'].notnull()]['text'].iloc[0]

values={'company':'zero','headline':'zero','positivity':0,'text':'zero','title':'zero','sentiment':0}
news=news.fillna(value=values)
news=news.drop('id', axis=1)
news.loc[(news.positivity >0) | (news.sentiment !=0)]
news['text']=news['text'].apply(cleanhtml)
news['text']=news['text'].apply(str.lower)
news['tokenized_text']=news['text'].apply(nltk.sent_tokenize)

positive_news=news.loc[(news.positivity >4) | (news.sentiment >0)]
positive_news=positive_news[['title','text','tokenized_text']]

negative_news=news.loc[((news.positivity >0) & (news.positivity <5) ) | (news.sentiment <0)]
negative_news=negative_news[['title','text','tokenized_text']]



positive_dic={}
negative_dic={}


 
#According to the formula of 
for i in range(len(positive_words)):
    result_positive=positive_news.loc[(positive_news['title'].str.contains(positive_words[i].lower())) | (positive_news['text'].str.contains(positive_words[i].lower()))]
    result_negative=negative_news.loc[(negative_news['title'].str.contains(positive_words[i].lower())) | (negative_news['text'].str.contains(positive_words[i].lower()))]
   
    if result_positive.empty is False:
        positive_dic[i]=(positive_words[i],float(Fraction(len(result_positive),len(result_positive)+len(result_negative))),float(Fraction(len(result_negative),len(result_positive)+len(result_negative))))


for i in range(len(negative_words)):
    result_positive=positive_news.loc[(positive_news['title'].str.contains(negative_words[i].lower())) | (positive_news['text'].str.contains(negative_words[i].lower()))]
    result_negative=negative_news.loc[(negative_news['title'].str.contains(negative_words[i].lower())) | (negative_news['text'].str.contains(negative_words[i].lower()))]
    
    if result_negative.empty is False:
        negative_dic[i]=(negative_words[i],float(Fraction(len(result_positive),len(result_positive)+len(result_negative))),float(Fraction(len(result_negative),len(result_positive)+len(result_negative))))

len(positive_dic),len(negative_dic)   

In [ ]:
def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext

In [ ]:
from pandas.plotting import table 


fig, ax = plt.subplots(figsize=(12, 2)) 
ax.xaxis.set_visible(False)  
ax.yaxis.set_visible(False)  
ax.set_frame_on(False) 
table = table(ax, headline, loc='upper right')  
table.auto_set_font_size(False) 
table.set_fontsize(12) 
table.scale(2, 2) 
table.auto_set_column_width(col=list(range(len(headline.columns))))
plt.savefig("headlineDataSet.png",dpi=400, bbox_inches = "tight")

In [ ]:
#Create a polarity score according to the formula in the following paper:
#Change structure of dicts into DataFrame for better visualization 
#This step is only necessary for visualization, the calculating of polarity_score can be done in the previous dictionary structure too


#CREATING DataFrame FOR POSITIVE WORDS
df=pd.DataFrame(positive_dic)
df=df.rename(columns=df.iloc[0])
df_positive=df.iloc[1:]
df_positive.loc[3]=2*(df_positive.iloc[0])-1

#Change STRUCTURE OF DATAFRAME and column names for better visualization
df_positive=df_positive.melt()
df_positive = df_positive.groupby('variable')['value'].apply(list).reset_index()       # groupby to list
df_positive = df_positive.join(pd.DataFrame(df_positive.pop('value').values.tolist()))  # expand lists to columns
df_positive=df_positive.rename(columns={'variable':'Word',0:'positiveSentScore',1:'negativeSentScore',2:'polarityScore'})
df_positive


#CREATING DataFrame FOR NEGATIVE WORDS
df_negative=pd.DataFrame(negative_dic)
df_negative=df_negative.rename(columns=df_negative.iloc[0])
df_negative=df_negative.iloc[1:]
df_negative.loc[3]=2*(df_negative.iloc[0])-1

#Change STRUCTURE OF DATAFRAME and column names for better visualization
df_negative=df_negative.melt()
df_negative = df_negative.groupby('variable')['value'].apply(list).reset_index()       # groupby to list
df_negative = df_negative.join(pd.DataFrame(df_negative.pop('value').values.tolist()))  # expand lists to columns
df_negative=df_negative.rename(columns={'variable':'Word',0:'positiveSentScore',1:'negativeSentScore',2:'polarityScore'})
df_negative,df_positive


#Combine both dataframes

df_own_lexicon = pd.concat([df_positive,df_negative])
df_own_lexicon.to_csv("/OwnLexiconPolarity.csv",index=False)
df_own_lexicon

In [ ]:
#Determine polarity_scores for the LM lexicon
from itertools import chain
from collections import defaultdict
postive_dict_LM={}
negative_dict_LM={}

for i in range(len(positive_words)):
    postive_dict_LM[i]=(positive_words[i],1)
    
for i in range(len(negative_words)):
    negative_dict_LM[i]=(negative_words[i],-1)

#Combine both dictionaries

#CREATING DataFrame FOR POSITIVE WORDS
dff=pd.DataFrame(postive_dict_LM)
dff=dff.rename(columns=dff.iloc[0])
dff_positive=dff.iloc[1:]
dff_positive

#Change STRUCTURE OF DATAFRAME and column names for better visualization
dff_positive=dff_positive.melt()
dff_positive=dff_positive.rename(columns={'variable':'Word','value':'polarityScore'})
dff_positive


#CREATING DataFrame FOR NEGATIVE WORDS
dfff=pd.DataFrame(negative_dict_LM)
dfff=dfff.rename(columns=dfff.iloc[0])
dfff_positive=dfff.iloc[1:]
dfff_positive

#Change STRUCTURE OF DATAFRAME and column names for better visualization
dfff_positive=dfff_positive.melt()
dfff_positive=dfff_positive.rename(columns={'variable':'Word','value':'polarityScore'})
dfff_positive


df_LM_lexicon = pd.concat([dff_positive,dfff_positive])
df_LM_lexicon.to_csv("LmLexiconPolarityNew.csv",index=False)
df_LM_lexicon
df_LM_lexicon['polarityScore']

In [ ]:
#Create a new dic with the necessary columns for sentiment analysis
own_dict=dict(zip(df_own_lexicon.Word.str.lower(), df_own_lexicon.polarityScore))
own_dict

#Create a new dic with the necessary columns for sentiment analysis
LM_dict=dict(zip(df_LM_lexicon.Word.str.lower(), df_LM_lexicon.polarityScore))
len(LM_dict)

In [ ]:
#Label the whole dataset according the chosen dictionary we created previously
#First method is for labelling single rows, second one is for all rows in a DataFrame
#Both method are explicitly created for LM and own created dictionaries

#label single Data in negative/positive polarity
def labelData(data,dictionary):
    polarity_result=0
    pol_list=[]
    for i in data:
        if dictionary.get(i) is not None :
            print(i,dictionary.get(i))
            pol_list.append(dictionary.get(i))
            polarity_result=sum(pol_list)/len(pol_list)
            
    return polarity_result    

#label all rows in a DataFrame into negative/positive DataFrames
def labelDataSet(data,dictionary,pathTo):
    row_counter=0
    polarity_result=0
    pol_list=[]
    while row_counter <len(data['name']): 
        for i in data['name'][row_counter]:
            if dictionary.get(i) is not None:
                pol_list.append(dictionary.get(i))
                polarity_result=sum(pol_list)/len(pol_list)
                data['numMentions'][row_counter]=polarity_result
        
        row_counter+=1
    return data.to_csv(pathTo)


In [ ]:
#Sentimentanalysis with SentWordNet, our third dictionary

from nltk import pos_tag
from nltk import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import word_tokenize, pos_tag
from collections import defaultdict
from nltk.corpus import sentiwordnet as swn
from nltk.corpus import wordnet as wn

    
def penn_to_wn(tag):
    if tag.startswith('J'):
        return wn.ADJ
    elif tag.startswith('N'):
        return wn.NOUN
    elif tag.startswith('R'):
        return wn.ADV
    elif tag.startswith('V'):
        return wn.VERB
    return None


def getSentimentWithSentiWordNet(data):
    words=data
    words=' '.join(words)
    tokens=word_tokenize(words)
    tags = pos_tag(tokens)
    sentiment = 0
    count = 0

    for word, tag in tags:
        wn_tag = penn_to_wn(tag)
        if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV):
            continue

        synsets = wn.synsets(word, pos=wn_tag)
        if not synsets:
            continue

        synset = synsets[0]
        swn_synset = swn.senti_synset(synset.name())
        word_sent = swn_synset.pos_score() - swn_synset.neg_score()
            
        if word_sent !=0:
            sentiment += word_sent
            count += 1
                
    if count >0:
        sentiment = sentiment/count

    return sentiment


#label all rows in a DataFrame into negative/positive DataFrames
def labelDataSentiWordNet(data,pathTo):
    row_counter=0
    polarity_result=0
    while row_counter <len(data['NameSet']): 
        polarity_result=getSentimentWithSentiWordNet(data['NameSet'][row_counter])
        data['numMentions'][row_counter]=polarity_result
        row_counter+=1
        print(f' Number {row_counter}')
    return data.to_csv(pathTo)

In [ ]:
#Facebook
COLORS = ["red", "orange", "green"]

# Drop the columns that aren't useful for the plot
plot_day = facebook.drop(['url', 'name'], axis=1)

# Change the column names to 'negative', 'positive', and 'neutral'
plot_day.columns = ['negative', 'positive', 'neutral']

# Plot a stacked bar chart
plot_day.plot(kind='bar', color=COLORS, figsize=(10,5), width=1)

In [ ]:
from gensim.summarization.bm25 import get_bm25_weights
import gensim.summarization.bm25 as bm

In [ ]:
#result = get_bm25_weights(master_file.iloc[0:10]['name'], n_jobs=-1)
#gbm.get_scores(master_file.iloc[0:1]['name'],[1])
Lexicon= pd.read_csv('OwnLexiconPolarity.csv')


a= bm.BM25(master_file.iloc[0:20192]['name'])
b=a.get_scores_bow('digest')


b[0]
    


In [ ]:
import collections
def addBM25():
    super_dict = collections.defaultdict(list)
    dic={}
    a= bm.BM25(master_file.iloc[0:20192]['name'])
    for i in query_tokens[0:955]:
        b=a.get_scores_bow(i)
        for i in b:
            key=i[0]
            value=i[1]
            dic[key]=value
        for k,v in dic.items():
            super_dict[k].append(v)
    return super_dict


In [ ]:
end_result=addBM25()

In [ ]:
result = {}
for k, v in end_result.items():
    result[k] = sum(v)

In [ ]:
def labelDataBM25(data,tokens):
    data=[word for word in data['name']]
    polarity_result=0
    result_list=[]
    count=0
    while count < len(data):
        for i in tokens:
            temp=a.get_scores(i)
            polarity_result+=temp[count]
        result_list.append(polarity_result)
        polarity_result=0
        count+=1
    return result_list
labelData

In [ ]:
from nltk import word_tokenize, pos_tag
s=pd.Series(Lexicon['Word'])
query=s.str.cat(sep=' ')
query=query.lower()
query_tokens=word_tokenize(query)
len(master_file)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
def flatten(x):
    result = []
    for el in x:
        if hasattr(el, "__iter__") and not isinstance(el, str):
            result.extend(flatten(el))
        else:
            result.append(el)
    return result

In [ ]:
df=master_file.iloc[0:20000]['name']
words=[ word for word in df if len(word)>2]


words=flatten(words)

top_N = 100
word_dist = nltk.FreqDist(words)
rslt = pd.DataFrame(word_dist.most_common(top_N),
                    columns=['Word', 'Frequency'])

plt.figure(figsize=(14,10))
sns.set_style("whitegrid")
ax = sns.barplot(x="Word",y="Frequency", data=rslt.head(12))


# PRAKTISCHE DURCHFÜHRUNG

In [ ]:

df_LM_lexicon=pd.read_csv("LmLexiconPolarityNeu.csv")
df_own_lexicon=pd.read_csv("OwnLexiconPolarityNeu.csv")


In [ ]:
df_own_lexicon.polarityScore.value_counts(bins=6),df_LM_lexicon.polarityScore.value_counts()

In [ ]:
df_own_lexicon['polarityScore'].mean()

In [ ]:
df_LM_lexicon['polarityScore'].mean()

In [ ]:
#Create a new dic with the necessary columns for sentiment analysis
own_dict=dict(zip(df_own_lexicon.Word.str.lower(), df_own_lexicon.polarityScore))
own_dict

#Create a new dic with the necessary columns for sentiment analysis
LM_dict=dict(zip(df_LM_lexicon.Word.str.lower(), df_LM_lexicon.polarityScore))
LM_dict


In [ ]:
import os
import pandas as pd
import glob

files = glob.glob(r'//*.csv')

df = pd.concat([pd.read_csv(f, index_col=[0,1]) for f in files])

In [ ]:
labelDataSentiWordNet(master_file,"masterFileSentiLexikon.csv")

In [ ]:
labelDataSet(master_file,LM_dict,"masterFileLmLexikonNeu.csv")

In [ ]:
labelDataSet(master_file,own_dict,"masterFileOwnLexikonNeu.csv")

In [ ]:
master_file_lm=pd.read_csv("masterFileLmLexikonNeu.csv")
#master_file_lm['numMentions'] = master_file_lm['numMentions'].apply(lambda x: float(x.split()[0].replace(',', '').replace('[',' ')))
master_file_lm=master_file_lm.drop('Unnamed: 0',axis=1)


In [ ]:
master_file_own_neu=pd.read_csv("masterFileOwnLexikonNeu.csv")
#master_file_own_neu['numMentions'] = master_file_own_neu['numMentions'].apply(lambda x: float(x.split()[0].replace(',', '').replace('[',' ')))
master_file_own_neu=master_file_own_neu.drop('Unnamed: 0',axis=1)

In [ ]:
master_file_senti=pd.read_csv("masterFileSentiLexikon.csv")
#master_file_senti['numMentions'] = master_file_senti['numMentions'].apply(lambda x: float(x.split()[0].replace(',', '').replace('[',' ')))
master_file_senti=master_file_senti.drop('Unnamed: 0',axis=1)

In [ ]:
len(master_file_lm[master_file_lm['numMentions']>0]),len(master_file_lm[master_file_lm['numMentions']<0])

In [ ]:
len(master_file_senti[master_file_senti['numMentions']>0]),len(master_file_senti[master_file_senti['numMentions']<0])

In [ ]:
len(master_file_own_neu[master_file_own_neu['numMentions']>0]),len(master_file_own_neu[master_file_own_neu['numMentions']<0])

# STEMMING

In [ ]:

import pandas as pd
from nltk.stem.snowball import SnowballStemmer

# Use English stemmer.
stemmer = SnowballStemmer("english")


# Make sure we see the full column.
#pd.set_option('display.max_colwidth', -1)

master_file['stemmed'] = master_file.iloc[0:34647]['name'].apply(lambda x: [stemmer.stem(y) for y in x]) # Stem every word.
master_file


master_file.to_csv("masterFileAllStemmedNeu.csv")


In [ ]:
master_file.to_csv("masterFileAllStemmed.csv")

In [ ]:

#LM-Dictionary has float values as keys, so we have to delete them
for k,w in LM_dict.items():
    if isinstance(k,(float)):
        LM_dict.pop(k,None)

In [ ]:

LM_dict_stemmed = {stemmer.stem(w) : LM_dict[w] for w in LM_dict.keys()}

In [ ]:

own_df_stemmed=pd.DataFrame.from_dict(own_dict_stemmed, orient='index')
LM_df_stemmed=pd.DataFrame.from_dict(LM_dict_stemmed, orient='index')

In [ ]:
own_df_stemmed.to_csv("OwnLexiconPolarityNeuStemmed.csv")
LM_df_stemmed.to_csv("LmLexiconPolarityStemmed.csv")

In [ ]:
master_file_stemmed=pd.read_csv("masterFileAllStemmedNeuNeu.csv")
master_file_stemmed['numMentions'] = master_file_stemmed['numMentions'].apply(lambda x: float(x.split()[0].replace(',', '').replace('[',' ')))
master_file_stemmed=master_file_stemmed.drop('Unnamed: 0',axis=1)

In [ ]:
own_df_stemmed=pd.read_csv("OwnLexiconPolarityNeuStemmed.csv")
LM_df_stemmed=pd.read_csv("LmLexiconPolarityStemmed.csv")

In [ ]:
own_df_stemmed = own_df_stemmed.rename(columns={'Unnamed: 0':'Word', '0': 'polarityScore'})
LM_df_stemmed=LM_df_stemmed.rename(columns={'Unnamed: 0':'Word', '0': 'polarityScore'})

In [ ]:
#Create a new dic with the necessary columns for sentiment analysis
own_dic_stemmed=dict(zip(own_df_stemmed.Word.str.lower(), own_df_stemmed.polarityScore))
own_dic_stemmed

#Create a new dic with the necessary columns for sentiment analysis
LM_dic_stemmed=dict(zip(LM_df_stemmed.Word.str.lower(), LM_df_stemmed.polarityScore))
LM_dic_stemmed

In [ ]:
#label all rows in a DataFrame into negative/positive DataFrames
def labelDataSet(data,dictionary,pathTo):
    row_counter=0
    polarity_result=0
    pol_list=[]
    while row_counter <len(data['stemmed']): 
        for i in data['stemmed'][row_counter]:
            if dictionary.get(i) is not None:
                pol_list.append(dictionary.get(i))
                polarity_result=sum(pol_list)/len(pol_list)
                data['numMentions'][row_counter]=polarity_result
        
        row_counter+=1
    return data.to_csv(pathTo)

In [ ]:
labelDataSet(master_file_stemmed,LM_dic_stemmed,"masterFileLMStemmedNeu.csv")

In [ ]:
master_file_stemmed_own=pd.read_csv("masterFileOwnStemmed.csv")
master_file_stemmed_own['numMentions'] = master_file_stemmed_own['numMentions'].apply(lambda x: float(x.split()[0].replace(',', '').replace('[',' ')))
master_file_stemmed_own=master_file_stemmed_own.drop('Unnamed: 0',axis=1)
master_file_stemmed_own['date'] = pd.to_datetime(master_file_stemmed_own.date).dt.date

In [ ]:
master_file_stemmed_LM_test=pd.read_csv("masterFileLMStemmed.csv")
#master_file_stemmed_LM['numMentions'] = master_file_stemmed_LM['numMentions'].apply(lambda x: float(x.split()[0].replace(',', '').replace('[',' ')))
master_file_stemmed_LM=master_file_stemmed_LM.drop('Unnamed: 0',axis=1)
master_file_stemmed_LM['date'] = pd.to_datetime(master_file_stemmed_LM.date).dt.date

In [ ]:
#master_file_stemmed_LM[master_file_stemmed_LM['stemmed'].str.contains('amazon')].groupby(['date']).sum()

In [ ]:
master_file_stemmed_LM.to_csv("masterFileLMStemmed.csv")

In [ ]:
#master_file_stemmed_own[master_file_stemmed_own['stemmed'].str.contains('amazon')].groupby(['date']).sum()

In [ ]:
LM_df_stemmed.polarityScore.value_counts(bins=3),own_df_stemmed.polarityScore.value_counts(bins=3)

In [ ]:
#own-lexikon-stemmed
stemmed_amazon=master_file_stemmed_own[master_file_stemmed_own['stemmed'].str.contains('amazon')].groupby(['date']).sum()
stemmed_google=master_file_stemmed_own[master_file_stemmed_own['stemmed'].str.contains('google')].groupby(['date']).sum()
stemmed_facebook=master_file_stemmed_own[master_file_stemmed_own['stemmed'].str.contains('facebook')].groupby(['date']).sum()
stemmed_apple=master_file_stemmed_own[master_file_stemmed_own['stemmed'].str.contains('apple')].groupby(['date']).sum()
stemmed_microsoft=master_file_stemmed_own[master_file_stemmed_own['stemmed'].str.contains('microsoft')].groupby(['date']).sum()



In [ ]:
#lm-lexikon-stemmed
stemmed_amazon_lm=master_file_stemmed_LM[master_file_stemmed_LM['stemmed'].str.contains('amazon')].groupby(['date']).sum()
stemmed_google_lm=master_file_stemmed_LM[master_file_stemmed_LM['stemmed'].str.contains('google')].groupby(['date']).sum()
stemmed_facebook_lm=stemmed_amazon_lm=master_file_stemmed_LM[master_file_stemmed_LM['stemmed'].str.contains('facebook')].groupby(['date']).sum()
stemmed_apple_lm=stemmed_amazon_lm=master_file_stemmed_LM[master_file_stemmed_LM['stemmed'].str.contains('apple')].groupby(['date']).sum()
stemmed_microsoft_lm=stemmed_amazon_lm=master_file_stemmed_LM[master_file_stemmed_LM['stemmed'].str.contains('microsoft')].groupby(['date']).sum()

In [ ]:
#stock-data

apple_data=pd.read_csv('AAPL.csv')
apple_data['endResult']=apple_data['Close']-apple_data['Open']

amazon_data=pd.read_csv('AMZN.csv')
amazon_data['endResult']=amazon_data['Close']-amazon_data['Open']

fb_data=pd.read_csv('FB.csv')
fb_data['endResult']=fb_data['Close']-fb_data['Open']

google_data=pd.read_csv('GOOG.csv')
google_data['endResult']=google_data['Close']-google_data['Open']

microsoft_data=pd.read_csv('MSFT.csv')
microsoft_data['endResult']=microsoft_data['Close']-microsoft_data['Open']

In [ ]:
#stock-data to excel
with pd.ExcelWriter('PredictionsStemmed.xlsx') as writer:  
    apple_data[['Date','endResult']].to_excel(writer, sheet_name='AppleStemmed')
    amazon_data[['Date','endResult']].to_excel(writer, sheet_name='AmazonStemmed')
    fb_data[['Date','endResult']].to_excel(writer, sheet_name='FbStemmed')
    google_data[['Date','endResult']].to_excel(writer, sheet_name='GoogleStemmed')
    microsoft_data[['Date','endResult']].to_excel(writer, sheet_name='MicrosoftStemmed')

In [ ]:

labelDataSet(master_file,LM_dic_stemmed,"masterFileLMStemmedNeuNeu.csv")

In [ ]:
master_file_stemmed_own=pd.read_csv("masterFileOwnStemmedNeu.csv")
master_file_stemmed_own['numMentions'] = master_file_stemmed_own['numMentions'].apply(lambda x: float(x.split()[0].replace(',', '').replace('[',' ')))
master_file_stemmed_own=master_file_stemmed_own.drop('Unnamed: 0',axis=1)
master_file_stemmed_own['date'] = pd.to_datetime(master_file_stemmed_own.date).dt.date

In [ ]:
master_file_stemmed_LM=pd.read_csv("masterFileLMStemmedNeuNeu.csv")
master_file_stemmed_LM['numMentions'] = master_file_stemmed_LM['numMentions'].apply(lambda x: float(x.split()[0].replace(',', '').replace('[',' ')))
master_file_stemmed_LM=master_file_stemmed_LM.drop('Unnamed: 0',axis=1)
master_file_stemmed_LM['date'] = pd.to_datetime(master_file_stemmed_LM.date).dt.date

In [ ]:
master_file_stemmed_LM[master_file_stemmed_LM['numMentions']<0]

# SET OF WORDS 

In [ ]:
master_file['union'] = master_file.iloc[0:2]['name'].apply(set(testing).union(set(testing)))

In [ ]:
master_file['NameSet'] = master_file.apply(lambda row: set(row['name']), axis=1)
master_file['StemmedSet'] = master_file.apply(lambda row: set(row['stemmed']), axis=1)

In [ ]:
master_file.to_csv("masterFileSetUnion.csv")

In [ ]:
master_file_union=pd.read_csv("masterFileSetUnion.csv")
master_file_union['numMentions'] = master_file_union['numMentions'].apply(lambda x: float(x.split()[0].replace(',', '').replace('[',' ')))
master_file_union=master_file_union.drop('Unnamed: 0',axis=1)
master_file_union['date'] = pd.to_datetime(master_file_union.date).dt.date

In [ ]:
labelDataSet(master_file,own_dict,"masterFileOwnSetUnionNeu.csv")

In [ ]:
labelDataSet(master_file,LM_dict,"masterFileLMSetUnionNeu.csv")

In [ ]:
labelDataSentiWordNet(master_file,"masterFileSentiSetUnionNeu.csv")

In [ ]:
master_file_set=pd.read_csv("masterFileOwnSetUnionNeu.csv")
master_file_set['numMentions'] = master_file_set['numMentions'].apply(lambda x: float(x.split()[0].replace(',', '').replace('[',' ')))
master_file_set=master_file_set.drop('Unnamed: 0',axis=1)
master_file_set['date'] = pd.to_datetime(master_file_set.date).dt.date

In [ ]:
master_file_set_LM=pd.read_csv("masterFileLMSetUnionNeu.csv")
master_file_set_LM['numMentions'] = master_file_set_LM['numMentions'].apply(lambda x: float(x.split()[0].replace(',', '').replace('[',' ')))
master_file_set_LM=master_file_set_LM.drop('Unnamed: 0',axis=1)
master_file_set_LM['date'] = pd.to_datetime(master_file_set_LM.date).dt.date

In [ ]:
master_file_set_senti=pd.read_csv("masterFileSentiSetUnionNeu.csv")
#master_file_set_senti['numMentions'] = master_file_set_senti['numMentions'].apply(lambda x: float(x.split()[0].replace(',', '').replace('[',' ')))
master_file_set_senti=master_file_set_senti.drop('Unnamed: 0',axis=1)
master_file_set_senti['date'] = pd.to_datetime(master_file_set_senti.date).dt.date

In [ ]:
#own-lexikon
amazon_set_own=master_file_set[master_file_set['NameSet'].str.contains('amazon')].groupby(['date']).sum()
apple_set_own=master_file_set[master_file_set['NameSet'].str.contains('apple')].groupby(['date']).sum()
google_set_own=master_file_set[master_file_set['NameSet'].str.contains('google')].groupby(['date']).sum()
microsoft_set_own=master_file_set[master_file_set['NameSet'].str.contains('microsoft')].groupby(['date']).sum()
fb_set_own=master_file_set[master_file_set['NameSet'].str.contains('facebook')].groupby(['date']).sum()

In [ ]:
#LM-Lexikon
amazon_set_lm=master_file_set_LM[master_file_set_LM['NameSet'].str.contains('amazon')].groupby(['date']).sum()
apple_set_lm=master_file_set_LM[master_file_set_LM['NameSet'].str.contains('apple')].groupby(['date']).sum()
google_set_lm=master_file_set_LM[master_file_set_LM['NameSet'].str.contains('google')].groupby(['date']).sum()
microsoft_set_lm=master_file_set_LM[master_file_set_LM['NameSet'].str.contains('microsoft')].groupby(['date']).sum()
fb_set_lm=master_file_set_LM[master_file_set_LM['NameSet'].str.contains('amazon')].groupby(['date']).sum()


In [ ]:
#sentiwordnet lexikon
amazon_set_senti=master_file_set_senti[master_file_set_senti['NameSet'].str.contains('amazon')].groupby(['date']).sum()
apple_set_senti=master_file_set_senti[master_file_set_senti['NameSet'].str.contains('apple')].groupby(['date']).sum()
google_set_senti=master_file_set_senti[master_file_set_senti['NameSet'].str.contains('google')].groupby(['date']).sum()
microsoft_set_senti=master_file_set_senti[master_file_set_senti['NameSet'].str.contains('microsoft')].groupby(['date']).sum()
fb_set_senti=master_file_set_senti[master_file_set_senti['NameSet'].str.contains('amazon')].groupby(['date']).sum()

In [ ]:
#apple_set_senti.groupby(['date']).sum()
#amazon_set_senti.groupby(['date']).sum()
#google_set_senti.groupby(['date']).sum()
#microsoft_set_senti.groupby(['date']).sum()
fb_set_senti.groupby(['date']).sum()

In [ ]:
#stock-data to excel
with pd.ExcelWriter('PredictionsSetUnion.xlsx') as writer:  
    apple_data[['Date','endResult']].to_excel(writer, sheet_name='AppleSet')
    amazon_data[['Date','endResult']].to_excel(writer, sheet_name='AmazonSet')
    fb_data[['Date','endResult']].to_excel(writer, sheet_name='FbSet')
    google_data[['Date','endResult']].to_excel(writer, sheet_name='GoogleSet')
    microsoft_data[['Date','endResult']].to_excel(writer, sheet_name='MicrosoftSet')

# Set Of words + Stemming

In [ ]:
labelDataSet(master_file,own_dict,"masterFileOwnSetUnionStemmedNeu.csv")

In [ ]:
labelDataSet(master_file,LM_dict,"masterFileLMSetUnionStemmedNeu.csv")

In [ ]:
master_file_set_stem=pd.read_csv("masterFileOwnSetUnionStemmedNeu.csv")
master_file_set_stem['numMentions'] = master_file_set_stem['numMentions'].apply(lambda x: float(x.split()[0].replace(',', '').replace('[',' ')))
master_file_set_stem=master_file_set_stem.drop('Unnamed: 0',axis=1)
master_file_set_stem['date'] = pd.to_datetime(master_file_set_stem.date).dt.date

In [ ]:
master_file_set_stem_LM=pd.read_csv("masterFileLMSetUnionStemmedNeu.csv")
master_file_set_stem_LM['numMentions'] = master_file_set_stem_LM['numMentions'].apply(lambda x: float(x.split()[0].replace(',', '').replace('[',' ')))
master_file_set_stem_LM=master_file_set_stem_LM.drop('Unnamed: 0',axis=1)
master_file_set_stem_LM['date'] = pd.to_datetime(master_file_set_stem_LM.date).dt.date

In [ ]:
#stock-data to excel
with pd.ExcelWriter('PredictionsSetUnionStemmed.xlsx') as writer:  
    apple_data[['Date','endResult']].to_excel(writer, sheet_name='AppleSetStem')
    amazon_data[['Date','endResult']].to_excel(writer, sheet_name='AmazonSetStem')
    fb_data[['Date','endResult']].to_excel(writer, sheet_name='FbSetStem')
    google_data[['Date','endResult']].to_excel(writer, sheet_name='GoogleSetStem')
    microsoft_data[['Date','endResult']].to_excel(writer, sheet_name='MicrosoftSetStem')

In [ ]:
#own-lexikon
amazon_set_stem_own=master_file_set_stem[master_file_set_stem['StemmedSet'].str.contains('amazon')].groupby(['date']).sum()
apple_set_stem_own=master_file_set_stem[master_file_set_stem['StemmedSet'].str.contains('apple')].groupby(['date']).sum()
fb_set_stem_own=master_file_set_stem[master_file_set_stem['StemmedSet'].str.contains('facebook')].groupby(['date']).sum()
google_set_stem_own=master_file_set_stem[master_file_set_stem['StemmedSet'].str.contains('google')].groupby(['date']).sum()
microsoft_set_stem_own=master_file_set_stem[master_file_set_stem['StemmedSet'].str.contains('microsoft')].groupby(['date']).sum()

In [ ]:
#LM-Lexikon
amazon_set_stem_lm=master_file_set_stem_LM[master_file_set_stem_LM['StemmedSet'].str.contains('amazon')].groupby(['date']).sum()
apple_set_stem_lm=master_file_set_stem_LM[master_file_set_stem_LM['StemmedSet'].str.contains('apple')].groupby(['date']).sum()
fb_set_stem_lm=master_file_set_stem_LM[master_file_set_stem_LM['StemmedSet'].str.contains('facebook')].groupby(['date']).sum()
google_set_stem_lm=master_file_set_stem_LM[master_file_set_stem_LM['StemmedSet'].str.contains('google')].groupby(['date']).sum()
microsoft_set_stem_lm=master_file_set_stem_LM[master_file_set_stem_LM['StemmedSet'].str.contains('microsoft')].groupby(['date']).sum()

In [ ]:
#microsoft_set_stem_own.groupby(['date']).sum()

# BM 25 Scoring

In [ ]:
from gensim.summarization.bm25 import get_bm25_weights
import gensim.summarization.bm25 as bm


In [ ]:
df_LM_lexicon=pd.read_csv("LmLexiconPolarityNeu.csv")
df_own_lexicon=pd.read_csv("OwnLexiconPolarityNeu.csv")

In [ ]:
#result = get_bm25_weights(master_file.iloc[0:10]['name'], n_jobs=-1)
#gbm.get_scores(master_file.iloc[0:1]['name'],[1])
Lexicon= pd.read_csv('OwnLexiconPolarity.csv')


a= bm.BM25(master_file_set_stem.iloc[0:34648]['StemmedSet'])
#b=a.get_scores_bow('digest')


#b[0]
    

    


    


In [ ]:
from nltk import word_tokenize, pos_tag
s=pd.Series(own_df_stemmed['Word'])
query=s.str.cat(sep=' ')
query=query.lower()
query_tokens=word_tokenize(query)
len(query_tokens)

In [ ]:
import collections
def addBM25():
    super_dict = collections.defaultdict(list)
    dic={}
    a= bm.BM25(master_file_set_stem.iloc[0:34648]['StemmedSet'])
    for i in query_tokens[0:947]:
        b=a.get_scores_bow(i)
        for i in b:
            key=i[0]
            value=i[1]
            dic[key]=value
        for k,v in dic.items():
            super_dict[k].append(v)
    return super_dict

In [ ]:

end_result=addBM25()

In [ ]:

result = {}
for k, v in end_result.items():
    result[k] = sum(v)

In [ ]:
    
masterFile_BM25_freq_own=master_file_set_stem
masterFile_BM25_freq_own["BM25"] = pd.Series(result)
#masterFile_BM25_freq_own=masterFile_BM25_freq_own.drop(columns='D',axis=1)
masterFile_BM25_freq_own=masterFile_BM25_freq_own.fillna(0)
masterFile_BM25_freq_own[masterFile_BM25_freq_own['numMentions']>0]


In [ ]:
#
masterFileBm25=pd.read_csv("masterFileBM25SetOwn.csv")
#masterFileBm25['numMentions'] = masterFileBm25['numMentions'].apply(lambda x: float(x.split()[0].replace(',', '').replace('[',' ')))
masterFileBm25=masterFileBm25.drop('Unnamed: 0',axis=1)
masterFileBm25['date'] = pd.to_datetime(masterFileBm25.date).dt.date

In [ ]:
#The algebraic sign of the Values that are negative in numMentions columns are being reflected on BM25 column values
masterFileBm25.loc[master_file_set_stem.numMentions<0,'BM25'] *= -1

In [ ]:
masterFileBm25[masterFileBm25['NameSet'].str.contains('microsoft')].groupby(['date']).sum()

In [ ]:
#stock-data to excel
with pd.ExcelWriter('PredictionsBM25SetStemmed.xlsx') as writer:  
    apple_data[['Date','endResult']].to_excel(writer, sheet_name='AppleSetStemmed')
    amazon_data[['Date','endResult']].to_excel(writer, sheet_name='AmazonSetStemmed')
    fb_data[['Date','endResult']].to_excel(writer, sheet_name='FbSetSetStemmed')
    google_data[['Date','endResult']].to_excel(writer, sheet_name='GoogleSetStemmed')
    microsoft_data[['Date','endResult']].to_excel(writer, sheet_name='MicrosoftSetStemmed')

# Results

In [ ]:
len(master_file_stemmed_LM[master_file_stemmed_LM['numMentions']>0]),len(master_file_stemmed_LM[master_file_stemmed_LM['numMentions']<0])

In [ ]:
#PredictionFreq Modell
AppleBm25LM=10/20
AppleBM25Own=12/20
AppleOwn=14/20
AppleLM=14/20
AppleSenti=11/20

GoogleLMBM25=11/20
GoogleBM25Own=11/20
GoogleLM=11/20
GoogleOwn=11/20
GoogleSenti=10/20

amazonbmLM=11/20
amazonbmOwn=12/20
AmazonLM=11/20
AmazonOwn=12/20
AmazonSenti=12/20

microsoftbmLM=9/20
microsftbmOwn=12/20
MicrosoftLM=15/20
MicrosoftOwn= 12/20
MicrosoftSenti=11/20

fbbmLM=10/20
fbbmown=11/20
facebookLM=8/20
facebookOwn=12/20
facebookSenti=10/20

#PredictionFreqStemmed
applebmLM=12/20
applebmOwn=11/20
AppleLM=12/20
AppleOwn=12/20

amazonbmLM=10/20
amazonbmOwn=9/20
AmazonLM=11/20
AmazonOwn=11/20

fbBmLM=10/20
fbBmOwn=9/20
FbLM=11/20
FbOwn=10/20

googlebmLM=9/20
googlebmOwn=10/20
GoogleLM=11/20
GoogleOwn=11/20

MicrosoftbmLM=10/20
microssftbmOwn=11/20
MicrosoftLM=11/20
MicrosoftOwn=11/20

#PredictionSet
applebmLM=11/20
applebmOwn=9/20
AppleLM=9/20
AppleOwn=11/20
AppleSenti=11/20

amazonbmLM=9/20
amazonbmOwn=9/120
AmazonLM=13/20
AmazonOwn=11/20
AmazonSenti=11/20

FbbmLM=9/20
FbbmOwn=9/20
FbLM=7/20
FbOwn=9/20
FbSenti=10/20

GooglebmLM=9/20
GooglebmOwn=9/20
GoogleLM=11/20
GoogleOwn=11/20
GoogleSenti=11/20

Microsoft=9/10
Microsftbmown=9/10
MicrosoftLM=15/20
MicrosoftOwn=11/20
Microsoft=11/20

#PredictionSetStemmed

ApplebmLM==11/20
ApplebmOwn=11/20
AppleLM=11/20
AppleOwn=12/20

AmazonbmLM=10/20
AmazonbmOwn=11/20
AmazonLM=14/20
AmazonOwn=11/20

FbBmLM=9/20
FbBmOwn=10/20
FbLM=10/20
FbOwn=10/20

googlebmLM=9/10
googlebmOwn=11/20
GoogleLM=12/20
GoogleOwn=11/20

microsoftbmLM=10/20
microsftbmOwn=11/20
MicrosoftLM=16/20
MicrosoftOwn=11/20


In [ ]:
#Amount of word in our lexicons after stemming it
((-1.003, -0.333]    3350
 (0.333, 1.0]        1277
 (-0.333, 0.333]        0
 Name: polarityScore, dtype: int64, (-0.333, 0.333]     364
 (0.333, 1.0]        313
 (-1.003, -0.333]    269
 Name: polarityScore, dtype: int64)

In [ ]:
own_df_stemmed['polarityScore'].mean()

In [ ]:
len(news.loc[(news.positivity >3) | (news.sentiment >0)]),len(news.loc[(news.positivity <3) | (news.sentiment <0)])

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
from wordcloud import WordCloud, STOPWORDS

text=' '.join(tokens)

# Generate a word cloud image
stop_words = ["google", "amazon", "apple", "facebook"] + list(STOPWORDS)
STOPWORDS.update(ignore_words)
wordcloud = WordCloud(stopwords=stop_words, background_color="white",width=1920, height=1080, max_words=100).generate(text)

# Display the generated image:
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()


In [ ]:
words=microsoft_data['name'][0:len(microsoft_data)]
words=[''.join(word) for word in words]
words=''.join(words)
tokens=word_tokenize(words)

In [ ]:
microsoft_data=master_file_union[master_file_union['name'].str.contains('microsoft')]